In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Neural Network

In [ ]:
class NeuralNetwork:
    W = []
    M = []
    b = []
    Yhat = []
    costs = []
    actf = 0
    
    def __sigmoid(self, Z):
        return 1 / (1 + np.exp(-Z))
    
    def __tanh(self, Z):
        return np.tanh(Z)
    
    def __relu(self,Z):
        return Z * (Z > 0)
    
    # Calls specified activation function
    def __sig(self,Z,ty):
        if ty == 0:
            return self.__sigmoid(Z)
        if ty == 1:
            return self.__tanh(Z)
        else:
            return self.__relu(Z)
        
        # Calls specified activation function derivative
    def __sig_dv(self,Z,ty):
        if ty == 0:
            return Z*(1-Z)
        if ty == 1:
            return (1-Z*Z)
        else:
            return np.where(Z > 0, 1, 0)
        
    def __softmax(self,A):
        expA = np.exp(A)
        return expA / expA.sum(axis=1,keepdims=True)
    
    # Cost for softmax
    def __cost(self,T,Y):
        tot = (-T * np.log(Y))
        return tot.sum()
    
    def predict(self,Py):
        return np.argmax(Py, axis=1)
    
    def classification_rate(self,T,Y):
        return np.mean(T == Y)
    
    # Adds hidden layer with L nodes
    def add_layer(self,L):
        NeuralNetwork.M.append(L)
    
    def __forward(self,M,W,b,actf):
        for i in range(1,len(W)+1):
            if (i != len(W)):
                M[i] = self.__sig((M[i-1].dot(W[i-1]) + b[i-1]), actf)
            else:
                M[i] = M[i-1].dot(W[i-1]) + b[i-1]
                Y = self.__softmax(M[i])
        return Y,M
    
    def fit(self,X,y,A,n_iter,learnR,reg):
        W = NeuralNetwork.W
        b = NeuralNetwork.b
        M = NeuralNetwork.M
        NeuralNetwork.actf = a
        
        # Initialize layers for M
        N = X.shape[0]
        for i in range(len(M)):
            M[i] = np.random.randn(N,M[i])
            
        k = len(set(y))
        M.insert(0,X)
        M.append(np.random.randn(N,K))
        l = learnR
        
        # Set indicator matrix
        Y = np.zeros((N,K))
        for i in range(N):
            Y[i,y[i]] = 1
            
        # Set weights
        for i in range(len(M)-1):
            if i == (len(M)-2):
                W.append(np.random.randn(M[i].shape[1],K) / np.sqrt(M[i].shape[1] + K))
                b.append(np.random.randn(K) / np.sqrt(K))
            else:
                W.append(np.random.randn(M[i].shape[1],M[i+1].shape[1]) / np.sqrt(M[i].shape[1] + M[i+1].shape[1]))
                b.append(np.random.randn(M[i+1].shape[1]) / np.sqrt(M[i+1].shape[1]))
                
        costs = []
        for e in range(n_iter):
            Yp,Z = self.__forward(M,W,b,a)
            cost = self.__cost(Y,Yp)
            costs.append(cost)
            if e % 1000 == 0:
                print(e,cost)
            
            # Adjust weights
            S = (Y - Yp)
            n = len(M)-2
            Zt = S
            
            for i in range(len(M)-1):
                W[n] += l * (Z[n].T.dot(Zt) - reg*W[n])
                b[n] += l * (Zt.sum() - reg*b[n])
                
                if i != (len(M)-2):
                    #Update Zt
                    Zt = Zt.dot(W[n].T)*self.__sig_dv(Z[n],a)
            
                n -= 1
        NeuralNetwork.W = W
        NeuralNetwork.b = b
        NeuralNetwork.M = M
        NeuralNetwork.Yhat = Yp
        NeuralNetwork.costs = costs
    
    def predict(self,X):
        W = NeuralNetwork.W
        b = NeuralNetwork.b
        M = NeuralNetwork.M
        a = NeuralNetwork.actf
        Yp,Z = self.__forward(M,W,b,a)
        NeuralNetwork.Yhat = Yp
        return Yp